In [1]:
from regulus.utils import io
from regulus.topo import * 
from regulus.alg import *
from regulus.measures import *
from regulus.models import *
from regulus.tree import *

from ipyregulus import TreeWidget, TreeView
from ipyregulus import SidePanel
from ipyregulus import *

In [2]:
gauss = io.load('data/gauss4.p')
gauss.add_attr('linear', node_model(linear_model))
gauss.add_attr('fitness', fitness)
gauss.add_attr('relative_fitness', relative_fitness)
gauss.tree.add_attr('parent_fitness', parent_fitness)
gauss.tree.add_attr('child_fitness', child_fitness)
gauss.tree.add_attr('size', node_size)
gauss.tree.add_attr('rel_size', node_relative_size)
gauss.tree.add_attr('span', node_span)

In [11]:
from ipywidgets import Output
debug_view = Output(title='debug')
debug_view

Output()

In [12]:
class TreeFilter(TreeControler):
    visible = Set(None, allow_none=True)

    def __init__(self, ref, attr, func, **kwargs):
        self._cbs = []
        super().__init__(ref, attr, func, **kwargs)
        self.tree = self.ref

    def ref_changed(self, change):
        super().ref_changed(change)
        self.tree = self.ref
        self.update(None)

    @debug_view.capture()
    def update(self, change):
        super().update(change)
        if self.ref is not None and self.monitored is not None:
            self.visible = filter_tree(self.ref, self.apply)
        else:
            self.visible = None
        for target, attr in self._cbs:
            print('assign', target, attr, self.visible)
            target[attr] = self.visible

    def connect(self, target, attr):
        self._cbs.append([target, attr])

    def on(self, func):
        self._cbs.append(func)

    def off(self, func):
        self._cbs.remove(func)

In [13]:
tree1 = gauss.tree
tree1

In [14]:
w1 = TreeWidget(tree=tree1)

TreeWidget.init <regulus.topo.regulus.RegulusTree object at 0x1138deef0>


In [15]:
w2 = TreeWidget(tree1)

TreeWidget.init <regulus.topo.regulus.RegulusTree object at 0x1138deef0>


In [16]:
v1 = TreeView(tree1, measure='fitness')

TreeWidget.init <regulus.topo.regulus.RegulusTree object at 0x1138deef0>


In [17]:
v1

TreeView(attrs={'fitness': {0: 0.37445782174461234, 1: 0.38560590369427816, 2: 0.43171894274965317, 3: 0.43591…

In [18]:
f1 = TreeFilter(w1, 'fitness', lambda a,b: a>=b)
f1

In [19]:
f1.connect(v1, 'show')

In [23]:
a = v1.show
type(a)

NoneType

In [11]:
def update(show):
    v1.show = show

In [12]:
f1.on(update)

In [13]:
f2 = TreeFilter(v1.tree, 'span',  lambda a,b: a>=b)

HasTree: <regulus.topo.regulus.RegulusTree object at 0x10f685d68>
HasTree.init assign ref= <regulus.topo.regulus.RegulusTree object at 0x10f685d68>


In [14]:
f2.tree

In [15]:
f1.value

0.42

In [16]:
f2

In [17]:
f2.on(update)

In [8]:
def test(h, *args, tree=None):
    print(h, ':', *args, 'tree', tree)

In [18]:
v2 = TreeView(w1)

HasTree: TreeWidget(root=<regulus.tree.tree.Node object at 0x10f685e10>)
HasTree.init assign ref= TreeWidget(root=<regulus.tree.tree.Node object at 0x10f685e10>)


In [19]:
v2

TreeView(field='start', tree_model=TreeWidget(root=<regulus.tree.tree.Node object at 0x10f685e10>))

In [21]:
v2.measure = 'rel_size'

In [22]:
f2.ref = v2.tree

In [23]:
v2.tree

In [24]:
f2.ref

In [25]:
f2.tree